In [1]:
import glob
import itertools
import json
import os
import pandas
import scipy
from scipy import stats
import matplotlib.pyplot as plt


In [2]:
path = "ExperimentData"
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

exp_paths = []

session_map = {}
for filename in dir_list:
    if filename.isdigit():
        exp_paths.append(path + "/" + filename + "/ResponsiveAR/Experiment/")
        #session_map[filename] = user_id
        files = os.listdir(path + "/" + filename + "/ResponsiveAR/Experiment/")
        name = files[0]
        index = name.find(".")
        user_id = name[index - 13: index]
        session_map[user_id] = int(filename)

print(exp_paths)

user_group_mapping = {}

for i in range(30, 87):
    user_group_mapping[i] = "A"
    if(i % 2 != 0):
        user_group_mapping[i] = "B"


    #special cases
    if(i == 79):
        user_group_mapping[i] = "A"

print(user_group_mapping)

Files and directories in ' ExperimentData ' :
['61', '59', '68', '57', '32', '35', '69', '56', '51', '58', '67', '60', '34', '33', '.DS_Store', '45', '80', '86', '44', '43', '38', '36', '31', '65', '54', '53', '30', '37', '39', '55', '63', '64', '46', '79', '41', '83', '48', '70', '84', '85', '71', '82', '49', '40', '47', '78']
['ExperimentData/61/ResponsiveAR/Experiment/', 'ExperimentData/59/ResponsiveAR/Experiment/', 'ExperimentData/68/ResponsiveAR/Experiment/', 'ExperimentData/57/ResponsiveAR/Experiment/', 'ExperimentData/32/ResponsiveAR/Experiment/', 'ExperimentData/35/ResponsiveAR/Experiment/', 'ExperimentData/69/ResponsiveAR/Experiment/', 'ExperimentData/56/ResponsiveAR/Experiment/', 'ExperimentData/51/ResponsiveAR/Experiment/', 'ExperimentData/58/ResponsiveAR/Experiment/', 'ExperimentData/67/ResponsiveAR/Experiment/', 'ExperimentData/60/ResponsiveAR/Experiment/', 'ExperimentData/34/ResponsiveAR/Experiment/', 'ExperimentData/33/ResponsiveAR/Experiment/', 'ExperimentData/45/Respon

In [3]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return float(fragments[1])

In [4]:
responsiveData = []
experimentEvents = []

def parseFrame(frames, user_id):
    frames_list = []
    for frame in frames:
        frameData = {
            "userID": float,
            "frameNum": int,
            "unixTime": float,
            "timestamp": float,
            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},
            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "experimentEvents": [],
            "responsiveData": []
        }
        responsiveEvent = frame["responsiveData"]
        experimentEvent = frame["experimentEvents"]
        frameData["userID"] = user_id
        frameData["frameNum"] = frame["frameNum"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]
        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["experimentEvents"] = experimentEvent
        frameData["responsiveData"] = responsiveEvent
        frames_list.append(frameData)
        if responsiveEvent:
            responsiveData.append(responsiveEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
    return frames_list

def parseSession(session, user_id):
    sessionData = {
    "userID": float,
    "numFrames": int,
    "task": str,
    "start_time": float,
    "end_time": float,
    "total_time": float,
    "sessionNumber": int,
    "isResponsive": int,
    "frames": []
    }
    sessionData["userID"] = user_id
    sessionData["numFrames"] = session["numFrames"]
    sessionData["task"] = session["task"]
    sessionData["start_time"] = session["start_time"]
    sessionData["end_time"] = session["end_time"]
    sessionData["total_time"] = session["total_time"]
    sessionData["sessionNumber"] = session["sessionNumber"]
    sessionData["isResponsive"] = session["isResponsive"]
    return sessionData


In [5]:
data = []
frames = []
allsession = []
user_id = 0

for exp in exp_paths:
    experiment_data = {
    "path" : str,
    "sessions": {}
    }
    sessions = {}
    experiment = os.listdir(exp)
    current_path = exp
    experiment_data["path"] = current_path
    experiment.sort(key=parseFileNumber)
    for filename in experiment:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            print(filename)
            firstFileFlag = False
            if("Session_0" in filename):
                firstFileFlag = True
            with open(current_path + filename, 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    user_id = session_map[json_data["obj"]["userID"]]
                    session = parseSession(json_data["obj"]["sessionRecordings"][0], user_id)
                    frame = parseFrame(json_data["obj"]["sessionRecordings"][0]["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
                else:
                    session = parseSession(json_data, user_id)
                    frame = parseFrame(json_data["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
    experiment_data["sessions"] = sessions
    data.append(experiment_data)

#print(data)


ExperimentSession_0_1653325550021.json
ExperimentSession_1_1653325550021.json
ExperimentSession_2_1653325550021.json
ExperimentSession_3_1653325550021.json
ExperimentSession_4_1653325550021.json
ExperimentSession_5_1653325550021.json
ExperimentSession_6_1653325550021.json
ExperimentSession_7_1653325550021.json
ExperimentSession_8_1653325550021.json
ExperimentSession_9_1653325550021.json
ExperimentSession_0_1652910130460.json
ExperimentSession_1_1652910130460.json
ExperimentSession_2_1652910130460.json
ExperimentSession_3_1652910130460.json
ExperimentSession_4_1652910130460.json
ExperimentSession_5_1652910130460.json
ExperimentSession_6_1652910130460.json
ExperimentSession_7_1652910130460.json
ExperimentSession_8_1652910130460.json
ExperimentSession_9_1652910130460.json
ExperimentSession_0_1653072453927.json
ExperimentSession_1_1653072453927.json
ExperimentSession_2_1653072453927.json
ExperimentSession_3_1653072453927.json
ExperimentSession_4_1653072453927.json
ExperimentSession_5_16530

In [6]:
allsession

sessionsUnpacked = list(itertools.chain(allsession))
sessionsDf = pandas.DataFrame(sessionsUnpacked)
sessionsDf = sessionsDf.sort_values(by='sessionNumber')

sessionsDf.head(1)

sessionsDf = sessionsDf.loc[sessionsDf["sessionNumber"] > 5]

USER 56 was unable to finish task 7 (when starting from 0) remove their task 8 from data
USER 85 was unable to finish task 7 and 8 (when starting from 0) remove their task 8 and 9 from data

In [18]:
sessionsDf.sort_values(by="sessionNumber")

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time
356,48,685,The task is to find the current temperature on...,1653430702402,1653430765713,63311,6,False,"[{'userID': 48, 'frameNum': 2295, 'unixTime': ...",1653430702412
156,80,551,The task is to find the current temperature on...,1653694177094,1653694221165,44071,6,False,"[{'userID': 80, 'frameNum': 1723, 'unixTime': ...",1653694177115
386,85,758,The task is to find the current temperature on...,1653699355491,1653699426792,71301,6,True,"[{'userID': 85, 'frameNum': 1119, 'unixTime': ...",1653699355501
434,47,197,The task is to find the current temperature on...,1653611166300,1653611178320,12020,6,True,"[{'userID': 47, 'frameNum': 1202, 'unixTime': ...",1653611166316
196,38,284,The task is to find the current temperature on...,1653593488627,1653593509077,20450,6,False,"[{'userID': 38, 'frameNum': 1480, 'unixTime': ...",1653593488636
...,...,...,...,...,...,...,...,...,...,...
109,67,1,,0,0,0,9,False,"[{'userID': 67, 'frameNum': 2617, 'unixTime': ...",0
129,34,1,,0,0,0,9,False,"[{'userID': 34, 'frameNum': 4089, 'unixTime': ...",0
69,69,1,,0,0,0,9,False,"[{'userID': 69, 'frameNum': 2353, 'unixTime': ...",0
299,63,1,,0,0,0,9,False,"[{'userID': 63, 'frameNum': 2780, 'unixTime': ...",0


In [7]:
framesUnpacked = list(itertools.chain(*sessionsDf["frames"]))
framesDf = pandas.DataFrame(framesUnpacked)
print(framesDf.shape)
framesDf.head(2)

(75552, 14)


,userID,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
0,48,2295,1653430693852,252274,"{'x': -0.7544376850128174, 'y': 0.052012041211...","{'x': 0.20998522639274597, 'y': 0.237312093377...","{'x': -0.11596442759037018, 'y': 0.11099042743...","{'x': 346.27203369140625, 'y': 12.483600616455...","{'x': -0.7537190318107605, 'y': 0.051911324262...","{'x': 0.21043619513511658, 'y': 0.241867095232...","{'x': -1.292854905128479, 'y': 3.6480093002319...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1653430693632, 'systemTime': '15...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."
1,48,2296,1653430693942,252364,"{'x': -0.7566391825675964, 'y': 0.052467077970...","{'x': 0.22454430162906647, 'y': 0.226982831954...","{'x': -0.11105890572071075, 'y': 0.11762709170...","{'x': 346.8804931640625, 'y': 13.3301753997802...","{'x': -0.7560760378837585, 'y': 0.052351504564...","{'x': 0.22062364220619202, 'y': 0.230719640851...","{'x': -1.1920766830444336, 'y': 3.538553476333...","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],"{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."


In [8]:
users = framesDf.loc[framesDf["experimentEvents"].str.len() != 0]
users.head(2)

,userID,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
0,48,2295,1653430693852,252274,"{'x': -0.7544376850128174, 'y': 0.052012041211...","{'x': 0.20998522639274597, 'y': 0.237312093377...","{'x': -0.11596442759037018, 'y': 0.11099042743...","{'x': 346.27203369140625, 'y': 12.483600616455...","{'x': -0.7537190318107605, 'y': 0.051911324262...","{'x': 0.21043619513511658, 'y': 0.241867095232...","{'x': -1.292854905128479, 'y': 3.6480093002319...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1653430693632, 'systemTime': '15...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."
86,48,2381,1653430702862,261284,"{'x': -0.025553148239850998, 'y': 0.0089262221...","{'x': 0.16106633841991425, 'y': -0.11054465919...","{'x': 0.05744396150112152, 'y': 0.079584002494...","{'x': 6.346714019775391, 'y': 9.32645988464355...","{'x': -0.024135341867804527, 'y': 0.0091597605...","{'x': 0.16187866032123566, 'y': -0.10814307630...","{'x': -0.4198451042175293, 'y': 2.250450134277...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1653430702412, 'systemTime': '15...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."


In [19]:
experimentUnpacked = list(itertools.chain(*users["experimentEvents"]))
experimentDf = pandas.DataFrame(experimentUnpacked)
experimentDf.head(1)
print(experimentDf.shape)

experimentDf["userID"] = users["userID"]

for i in users.index:
    eventDF = pandas.DataFrame(users["experimentEvents"][i])

    for j in eventDF.index:
        time = eventDF["unixTime"][j]
        userID = users["userID"][i]
        mask = experimentDf["unixTime"] == time
        experimentDf.loc[mask, "userID"] = userID
        unixTime = users["unixTime"][i]

experimentDf.head(2)

experimentDf = experimentDf.loc[experimentDf["task_number"] > 5]

(2483, 10)


In [20]:
experimentDf.shape[0]

2393

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1653430693632,15-18-13-63,False,answer,5,Keypad,"{'x': 0.8999999761581421, 'y': 0.8999999761581...","{'x': -0.5856979489326477, 'y': 0.071303784847...",8058272338,8058272338,48.0
160,1652737072112,14-37-52-11,False,answer,5,Keypad,"{'x': 0.8999999761581421, 'y': 0.8999999761581...","{'x': -0.295993447303772, 'y': 0.0565610602498...",8058272338,8058272338,32.0
161,1652737072122,14-37-52-12,False,complete,5,Keypad,"{'x': 0.8999999761581421, 'y': 0.8999999761581...","{'x': -0.295993447303772, 'y': 0.0565610602498...",8058272338,8058272338,32.0
165,1652910302772,14-45-02-77,True,answer,5,Keypad,"{'x': 0.8999999761581421, 'y': 0.8999999761581...","{'x': -0.37671542167663574, 'y': 0.03245545551...",8058272338,8058272338,59.0
166,1652910302772,14-45-02-77,True,complete,5,Keypad,"{'x': 0.8999999761581421, 'y': 0.8999999761581...","{'x': -0.37671542167663574, 'y': 0.03245545551...",8058272338,8058272338,59.0
...,...,...,...,...,...,...,...,...,...,...,...
1977,1653422937252,13-08-57-25,False,answer,8,Weather,"{'x': 0.7699999809265137, 'y': 0.7699999809265...","{'x': -0.02338077872991562, 'y': 0.00432424992...",51,6,64.0
1978,1653422939876,13-08-59-87,False,answer,8,Weather,"{'x': 0.7699999809265137, 'y': 0.7699999809265...","{'x': -0.02338077872991562, 'y': 0.00432424992...",51,62,64.0
1979,1653422948491,13-09-08-49,False,answer,8,Weather,"{'x': 0.7699999809265137, 'y': 0.7699999809265...","{'x': -0.02338077872991562, 'y': 0.00432424992...",51,6,64.0
1970,1653422894999,13-08-14-99,False,answer,8,Weather,"{'x': 0.4300000071525574, 'y': 0.4300000071525...","{'x': 0.037872254848480225, 'y': 0.03566391393...",isla vista,,64.0


In [21]:
startEvents = experimentDf.loc[experimentDf["eventName"] == "start"]
startEvents.head(3)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
3,1653430702412,15-18-22-41,False,start,6,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.750862181186676, 'y': 0.1321426033973...",76,,48.0
21,1652743322967,16-22-02-96,True,start,6,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.4349519610404968, 'y': 0.102766558527...",76,,33.0
26,1653330207911,11-23-27-91,True,start,6,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': 0.042348772287368774, 'y': 0.11162251979...",76,,83.0


In [22]:
sessionsDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time
356,48,685,The task is to find the current temperature on...,1653430702402,1653430765713,63311,6,False,"[{'userID': 48, 'frameNum': 2295, 'unixTime': ...",1653430702412
136,33,178,The task is to find the current temperature on...,1652743322957,1652743334928,11971,6,True,"[{'userID': 33, 'frameNum': 857, 'unixTime': 1...",1652743322967
346,83,1020,The task is to find the current temperature on...,1653330207901,1653330312033,104132,6,True,"[{'userID': 83, 'frameNum': 915, 'unixTime': 1...",1653330207911
166,86,509,The task is to find the current temperature on...,1653505210400,1653505255111,44711,6,False,"[{'userID': 86, 'frameNum': 1959, 'unixTime': ...",1653505210420
216,31,1002,The task is to find the current temperature on...,1652726277353,1652726381484,104131,6,True,"[{'userID': 31, 'frameNum': 2525, 'unixTime': ...",1652726277364
...,...,...,...,...,...,...,...,...,...,...
369,70,1,,0,0,0,9,False,"[{'userID': 70, 'frameNum': 2210, 'unixTime': ...",0
119,60,1,,0,0,0,9,False,"[{'userID': 60, 'frameNum': 3456, 'unixTime': ...",0
239,54,1,,0,0,0,9,False,"[{'userID': 54, 'frameNum': 2705, 'unixTime': ...",0
39,57,1,,0,0,0,9,False,"[{'userID': 57, 'frameNum': 3809, 'unixTime': ...",0


In [23]:
sessionsDf["Actual_Start_Time"] = sessionsDf["start_time"]

for i in startEvents.index:
    userID = startEvents["userID"][i]
    task_num = startEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_Start_Time"] = startEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time
356,48,685,The task is to find the current temperature on...,1653430702402,1653430765713,63311,6,False,"[{'userID': 48, 'frameNum': 2295, 'unixTime': ...",1653430702412


In [24]:
endEvents = experimentDf.loc[(experimentDf["eventName"] == "answer") & (experimentDf["guess"] == experimentDf["correct_answer"])]
endEvents.head(1)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
449,1653606268100,16-04-28-10,False,answer,6,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.41601043939590454, 'y': 0.10122574120...",62,62,84.0


In [25]:
sessionsDf["Actual_End_Time"] = sessionsDf["end_time"]

for i in endEvents.index:
    userID = endEvents["userID"][i]
    task_num = endEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_End_Time"] = endEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time
356,48,685,The task is to find the current temperature on...,1653430702402,1653430765713,63311,6,False,"[{'userID': 48, 'frameNum': 2295, 'unixTime': ...",1653430702412,1653430765703


In [26]:
end_time_diff = sessionsDf["end_time"].to_numpy() - sessionsDf["Actual_End_Time"].to_numpy()
end_time_diff

array([10,  0,  0, 10,  9, 10, 10,  9, 10, 11,  0, 10, 10, 11,  9,  0, 10,
       10, 10, 10,  5,  0, 10,  0, 11,  0, 10,  0,  0,  9, 10, 10, 10, 10,
        9,  0,  6, 10, 11,  0,  9, 11,  1, 10, 10, 10,  0,  7,  0, 11, 10,
       10,  0,  7, 10,  4,  9,  0, 11,  0,  0,  0,  0, 11, 10,  0,  8,  9,
        0,  0, 10, 10,  5, 11,  9,  9, 10, 11, 10, 10, 10, 10,  0,  0, 10,
        5, 10, 10,  0,  0,  0, 10, 15,  4,  0, 10,  0, 15,  0, 20,  9,  0,
        0, 13,  9,  9,  5,  5,  9, 10,  0,  0,  9,  0, 11,  0,  1, 16, 10,
        5, 10,  0,  0, 10, 10, 10, 11,  4, 10,  0,  9,  9,  9, 10,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0])

In [29]:
#first correct guess on keyboard
guessEvents = experimentDf.loc[experimentDf["eventName"] == "answer"]
guessEvents.head(3)

sessionsDf["Inital_Guess"] = sessionsDf["end_time"]

guessEvents = guessEvents.sort_values(by=["userID", "unixTime"])

guessEvents

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
302,1652721984488,10-26-24-48,False,answer,6,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.4405974745750427, 'y': 0.093236640095...",76,6,30.0
1331,1652721985538,10-26-25-53,False,answer,6,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.4405974745750427, 'y': 0.093236640095...",63,63,30.0
1333,1652721985588,10-26-25-58,False,answer,7,Weather,"{'x': 0.30000001192092896, 'y': 0.300000011920...","{'x': -0.4405974745750427, 'y': 0.093236640095...",63,,30.0
1336,1652722007318,10-26-47-31,False,answer,7,Weather,"{'x': 0.4300000071525574, 'y': 0.4300000071525...","{'x': -0.3796716332435608, 'y': 0.020988389849...",isla vista,i,30.0
1337,1652722008318,10-26-48-31,False,answer,7,Weather,"{'x': 0.4300000071525574, 'y': 0.4300000071525...","{'x': -0.3796716332435608, 'y': 0.020988389849...",isla vista,is,30.0
...,...,...,...,...,...,...,...,...,...,...,...
1489,1653505293602,12-01-33-60,False,answer,7,Weather,"{'x': 0.4300000071525574, 'y': 0.4300000071525...","{'x': 0.024460427463054657, 'y': 0.02306915074...",isla vista,isla vist,86.0
2206,1653505294952,12-01-34-95,False,answer,7,Weather,"{'x': 0.4300000071525574, 'y': 0.4300000071525...","{'x': 0.024460427463054657, 'y': 0.02306915074...",isla vista,isla vista,86.0
2208,1653505295032,12-01-35-03,False,answer,8,Weather,"{'x': 0.4300000071525574, 'y': 0.4300000071525...","{'x': 0.024460427463054657, 'y': 0.02306915074...",isla vista,,86.0
2211,1653505315452,12-01-55-45,False,answer,8,Weather,"{'x': 0.7699999809265137, 'y': 0.7699999809265...","{'x': 0.03459175303578377, 'y': -0.00459708273...",54,5,86.0


In [ ]:
for i in guessEvents.index:
    userID = guessEvents["userID"][i]
    task_num = guessEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Inital_Guess"] = guessEvents["unixTime"][i]

sessionsDf.head(1)

In [236]:
guessEvents = experimentDf.loc[experimentDf["eventName"] == "answer"]
guessEvents.head(3)

sessionsDf["Guesses"] = 0

In [237]:
for i in guessEvents.index:
    userID = guessEvents["userID"][i]
    task_num = guessEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Guesses"] += 1

In [238]:
start_time_diff = sessionsDf["start_time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
start_time_diff

array([           -50,            -60,            -50,            -50,
                  -50,            -60,            -50,            -56,
                  -50,            -50,            -50,            -61,
                  -50,            -50,            -60,            -56,
                  -50,            -61,            -60,            -51,
                  -60,            -50,            -49,            -52,
                  -62,            -60,            -59,            -50,
                  -60,            -49,            -51,            -61,
                  -50,            -50,            -50,            -50,
                  -43,            -60,            -60,            -45,
                  -60,            -50,            -60,            -51,
                  -60,            -45,            -40,            -30,
                  -40,            -40,            -43,            -40,
                  -50,            -40,            -38,            -39,
      

In [239]:
sessionsDf["Actual_Total_Time"] = sessionsDf["Actual_End_Time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575792,1,2810


In [240]:
start_time_diff = sessionsDf["total_time"].to_numpy() - sessionsDf["Actual_Total_Time"].to_numpy()
start_time_diff

array([           59,            64,            50,            50,
                  60,            60,            60,            56,
                  60,            60,            66,            71,
                  67,            50,            70,            65,
                  60,            61,            70,            51,
                  66,            50,            49,            62,
                  66,            60,            59,            60,
                  70,            49,            53,            61,
                  60,            50,            60,            50,
                  43,            60,            70,            52,
                  70,            60,            72,            62,
                  60,            60,            50,            40,
                  51,            40,            43,            50,
                  50,            40,            38,            49,
                  40,            49,            50,           

In [241]:
allsessionsDf = sessionsDf.loc[sessionsDf['sessionNumber'] < 9]
allsessionsDf.to_excel("sessionsData.xlsx")

In [242]:
dataFrame = pandas.DataFrame(columns=['userID', 'Mode', 'Application', 'Time', 'Task', 'Guesses', 'Avg Dist', 'STD Dist', 'Avg Scale', 'Avg Scale:Dist', 'Start Dist'])

ordered = sessionsDf.sort_values(by=["userID","sessionNumber"])
dataFrame['userID'] = ordered["userID"]
dataFrame['Task'] = ordered["sessionNumber"] + 1
dataFrame['Guesses'] = ordered["Guesses"]
dataFrame['Mode'] = ordered['isResponsive']
dataFrame['Time'] = ordered['Actual_Total_Time']

dataFrame = dataFrame.loc[dataFrame["Task"] < 10]

#Removing weather tasks for user 56 and 85, were unable to complete them
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 56) & (dataFrame["Task"] == 7)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 56) & (dataFrame["Task"] == 8)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 56) & (dataFrame["Task"] == 9)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 85) & (dataFrame["Task"] == 7)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 85) & (dataFrame["Task"] == 8)]
dataFrame = dataFrame.drop(findRowToDrop.index)
findRowToDrop = dataFrame.loc[(dataFrame["userID"] == 85) & (dataFrame["Task"] == 9)]
dataFrame = dataFrame.drop(findRowToDrop.index)

In [243]:
sessionsDf.head(2)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575792,1,2810
120,34,210,The task is to click on the 2nd article title....,1652747526667,1652747540607,13940,0,False,"[{'userID': 34, 'frameNum': 1, 'unixTime': 165...",1652747526727,1652747540603,1,13876


In [244]:
import math
import ast
def calcDistance(hPos, aPos):
    xDir = hPos['x'] - aPos['x']
    yDir = hPos['y'] - aPos['y']
    zDir = hPos['z'] - aPos['z']
    
    return math.sqrt(xDir*xDir + yDir*yDir + zDir*zDir)

for i in dataFrame.index:
    userID = dataFrame["userID"][i]
    task = dataFrame["Task"][i] - 1
    responsive = dataFrame['Mode'][i]

    findDistForSession = sessionsDf.loc[sessionsDf["userID"] == userID]

    startEvent = startEvents.loc[(startEvents["userID"] == userID) & (startEvents["task_number"] == task)]
    startTime = startEvent["unixTime"].to_numpy()[0]
    endEvent = endEvents.loc[(endEvents["userID"] == userID) & (endEvents["task_number"] == task)]
    endTime = endEvent["unixTime"].to_numpy()[0]
    framesUnpacked = list(itertools.chain(*findDistForSession["frames"]))
    sessionFramesDf = pandas.DataFrame(framesUnpacked)

    sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] >= startTime]
    #sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] <= endTime + 100]
    sessionFramesDf = sessionFramesDf.sort_values("unixTime")

    lastFrame = sessionFramesDf.loc[sessionFramesDf["experimentEvents"].str.len() != 0]
    lastFrame = lastFrame.loc[lastFrame["unixTime"] >= endTime]
    lastFrame = lastFrame.loc[lastFrame["unixTime"] <= endTime + 600]

    if(lastFrame.shape[0] != 1):
        print(endTime)
        print(lastFrame.shape[0])
        print(lastFrame["unixTime"].to_numpy())

    lastFrameTime = lastFrame["unixTime"].to_numpy()[0]
    sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] <= lastFrameTime]

    if(responsive):
        responsiveData = pandas.DataFrame(list(itertools.chain(sessionFramesDf["responsiveData"])))
        dist_avg = responsiveData['distance'].mean()
        dist_std = responsiveData['distance'].std()
        scales = pandas.DataFrame(list(itertools.chain(responsiveData["scale"])))
        scales = scales.loc[scales['x'] != 0]
        scale_avg = scales['x'].mean()

    else:
        Positions = pandas.DataFrame(columns=['hPos', 'aPos', 'unixTime', 'distance', 'scale'])
        Positions["hPos"] = sessionFramesDf["hPos"]
        Positions["unixTime"] = sessionFramesDf["unixTime"]
        experimentEventsForTask = pandas.DataFrame(list(itertools.chain(*sessionFramesDf["experimentEvents"])))
        experimentEventsForTask = experimentEventsForTask.loc[experimentEventsForTask["task_number"] == task]
        for t in experimentEventsForTask.index:
            event_time = experimentEventsForTask["unixTime"][t]
            mask = (sessionFramesDf['unixTime'] >= event_time)
            objectScale = {}
            objectPos = {}
            objectPos = experimentEventsForTask["object_position"][t]
            objectScale = experimentEventsForTask["object_scale"][t]
            Positions.loc[mask, 'aPos'] = str(objectPos)
            Positions.loc[mask, 'scale'] = objectScale['x']

        for j in Positions.index:
            hPos = Positions["hPos"][j]
            aPos = Positions["aPos"][j]
            aPos = ast.literal_eval(aPos)
            mask = Positions.index == j
            Positions.loc[mask, 'distance'] = calcDistance(hPos, aPos)

        dist_avg = Positions['distance'].mean()
        dist_std = Positions['distance'].std()
        scale_avg = Positions['scale'].mean()

    mask = dataFrame.index == i
    dataFrame.loc[mask, 'Avg Dist'] = dist_avg
    dataFrame.loc[mask, 'STD Dist'] = dist_std
    dataFrame.loc[mask, 'Avg Scale'] = scale_avg
    dataFrame.loc[mask, 'Avg Scale:Dist'] = scale_avg/dist_avg

1653088273395
2
[1653088273475 1653088273736]


In [245]:
#converting from task number to application and sub task number

mask = dataFrame['Task'] <= 4
dataFrame.loc[mask, 'Application'] = "Article"

mask = dataFrame['Task'] == 5
dataFrame.loc[mask, 'Task'] = 1
dataFrame.loc[mask, 'Application'] = "Numpad"

mask = dataFrame['Task'] == 6
dataFrame.loc[mask, 'Task'] = 2

dataFrame.loc[mask, 'Application'] = "Numpad"

mask = dataFrame['Task'] == 7
dataFrame.loc[mask, 'Task'] = 1
dataFrame.loc[mask, 'Application'] = "Weather"

mask = dataFrame['Task'] == 8
dataFrame.loc[mask, 'Task'] = 2
dataFrame.loc[mask, 'Application'] = "Weather"

mask = dataFrame['Task'] == 9
dataFrame.loc[mask, 'Task'] = 3

dataFrame.loc[mask, 'Application'] = "Weather"

In [246]:
mask = dataFrame['Mode'] == False
dataFrame.loc[mask, 'Mode'] = "Static"

mask = dataFrame['Mode'] == True
dataFrame.loc[mask, 'Mode'] = "Responsive"

dataFrame

,userID,Mode,Application,Time,Task,Guesses,Avg Dist,STD Dist,Avg Scale,Avg Scale:Dist,Start Dist
250,30,Static,Article,17181,1,1,0.650775,0.0267761,0.25,0.384157,NaN
251,30,Static,Article,12720,2,1,0.606882,0.011124,0.25,0.411942,NaN
252,30,Static,Article,17170,3,1,0.622351,0.055196,0.35,0.562383,NaN
253,30,Static,Article,9960,4,1,0.556536,0.117192,0.55,0.988256,NaN
254,30,Static,Numpad,21691,1,1,0.456405,0.14284,0.55,1.20507,NaN
...,...,...,...,...,...,...,...,...,...,...,...
164,86,Static,Numpad,47231,1,1,0.558575,0.06417,0.55,0.984649,NaN
165,86,Static,Numpad,23550,2,1,0.576774,0.0153846,0.9,1.5604,NaN
166,86,Static,Weather,44681,1,2,0.593761,0.0970801,0.3,0.505254,NaN
167,86,Static,Weather,30271,2,11,0.671105,0.01976,0.43,0.640734,NaN


In [247]:
dataFrame = dataFrame.sort_values(by= ['Application', 'Task', 'Mode', 'Time'])
dataFrame.to_csv("ExperimentData.csv")

articleDf = dataFrame.loc[dataFrame["Application"] == "Article"]
articleDf = articleDf.sort_values(by= ['Task', 'Mode', 'Time'])
articleDf.to_csv("ArticleTasksExperimentData.csv")
numpadDf = dataFrame.loc[dataFrame["Application"] == "Numpad"]
numpadDf = numpadDf.sort_values(by= ['Task', 'Mode', 'Time'])
numpadDf.to_csv("NumpadTasksExperimentData.csv")
weatherDf = dataFrame.loc[dataFrame["Application"] == "Weather"]
weatherDf = weatherDf.sort_values(by= ['Task', 'Mode', 'Time'])
weatherDf.to_csv("WeatherTasksExperimentData.csv")

In [248]:
for i in startEvents.index:
    user_id = startEvents["userID"][i]
    task = startEvents["task_number"][i]
    endEvent = endEvents.loc[(endEvents["userID"] == user_id) & (endEvents["task_number"] == task)]
    initialScale = startEvents["object_scale"][i]
    initialPos = startEvents["object_position"][i]
    endScale = endEvent["object_scale"].to_numpy()[0]
    endPos = endEvent["object_position"].to_numpy()[0]
    if(initialScale != endScale):
        print(f"scale before {initialScale}")
        print(f"scale after {endScale}")
        print(user_id)
        print(task)
        print(endEvent["isResponsive"])

    if(initialPos != endPos):
        print(f"pos before {initialPos}")
        print(f"pos after {endPos}")
        print(user_id)
        print(task)
        print(endEvent["isResponsive"])


pos before {'x': -0.011380864307284355, 'y': 0.025369271636009216, 'z': 0.6889793276786804}
pos after {'x': 0.01829620823264122, 'y': -0.08764401823282242, 'z': 0.5361096262931824}
82.0
0
230    False
Name: isResponsive, dtype: bool
pos before {'x': 0.07098753750324249, 'y': 0.042272619903087616, 'z': 0.12684473395347595}
pos after {'x': -0.015814445912837982, 'y': 0.058153215795755386, 'z': 0.14182975888252258}
56.0
1
476    False
Name: isResponsive, dtype: bool
scale before {'x': 0.25, 'y': 0.25, 'z': 0.25}
scale after {'x': 0.2609071433544159, 'y': 0.2609071433544159, 'z': 0.2609071433544159}
46.0
1
425    False
Name: isResponsive, dtype: bool
pos before {'x': -0.014312457293272018, 'y': 0.013834120705723763, 'z': 0.6040394902229309}
pos after {'x': -0.02931191399693489, 'y': -0.000485675991512835, 'z': 0.605438232421875}
46.0
1
425    False
Name: isResponsive, dtype: bool
scale before {'x': 0.25, 'y': 0.25, 'z': 0.25}
scale after {'x': 0.7625182271003723, 'y': 0.7625182271003723, '

IndexError: index 0 is out of bounds for axis 0 with size 0

In [249]:
article =  dataFrame.loc[dataFrame["Application"] == "Article"]
for i in range(1,5):
    taskDf = article.loc[article["Task"] == i]
    task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
    task_n = taskDf.loc[taskDf["Mode"] == "Static"]

    print(f"ARTICLE TASK {i}: ")

    taskRAvgTime = task_r['Time'].mean()
    taskRStd = task_r['Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Time'].mean()
    taskNStd = task_n['Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))
    
    taskRAvgTime = task_r['Guesses'].mean()
    taskRStd = task_r['Guesses'].std()
    print(f"responsive task {str(i)} Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Guesses'].mean()
    taskNStd = task_n['Guesses'].std()
    print(f"normal task {str(i)} Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

    taskRAvgTime = task_r['Avg Dist'].mean()
    taskRStd = task_r['Avg Dist'].std()
    print(f"responsive task {str(i)} Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Avg Dist'].mean()
    taskNStd = task_n['Avg Dist'].std()
    print(f"normal task {str(i)} Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))

ARTICLE TASK 1: 
responsive task 1 Time: 5868.522 ± 2744.399
normal task 1 Time: 8721.682 ± 9498.250
Ttest_indResult(statistic=-1.3821953988748257, pvalue=0.17404979230459425)
responsive task 1 Guesses: 1.043 ± 0.209
normal task 1 Guesses: 1.091 ± 0.294
Ttest_indResult(statistic=-0.626117235046913, pvalue=0.5345477403968492)
responsive task 1 Dist: 0.611 ± 0.043
normal task 1 Dist: 0.624 ± 0.044
Ttest_indResult(statistic=-0.9907393823025629, pvalue=0.32735539311446293)
ARTICLE TASK 2: 
responsive task 2 Time: 5195.391 ± 3871.828
normal task 2 Time: 13861.909 ± 12837.685
Ttest_indResult(statistic=-3.0951827958607496, pvalue=0.003453951553728935)
responsive task 2 Guesses: 1.130 ± 0.458
normal task 2 Guesses: 1.500 ± 0.859
Ttest_indResult(statistic=-1.8121843206506378, pvalue=0.0769443111920115)
responsive task 2 Dist: 0.617 ± 0.061
normal task 2 Dist: 0.599 ± 0.063
Ttest_indResult(statistic=0.9531813470525867, pvalue=0.3458259217371732)
ARTICLE TASK 3: 
responsive task 3 Time: 15482.348

In [250]:

articleTasks = pandas.DataFrame(columns=['userID', 'Mode', 'Application', 'Time', 'Guesses', 'Avg Dist', 'Avg Ratio' ])
articleTasks["userID"] = article["userID"]
articleTasks["Mode"] = article["Mode"]
articleTasks = articleTasks.drop_duplicates(subset=["userID"], keep="first")
articleTasks = articleTasks.sort_values(by='Time')

for i in articleTasks.index:
    userID = articleTasks["userID"][i]
    total_time = 0
    user = article.loc[article["userID"] == userID]
    total_time = sum(user["Time"])
    total_guesses = sum(user["Guesses"])

    articleTasks["Time"][i] = total_time
    articleTasks["Guesses"][i] = total_guesses
    articleTasks["Application"][i] = "Article"


for i in articleTasks.index:
    userID = articleTasks["userID"][i]
    user = article.loc[article["userID"] == userID]
    total_time = articleTasks["Time"][i]
    avg_dist = 0
    avg_ratio = 0
    for j in user.index:
        task_time = user["Time"][j]
        weight = task_time/total_time
        avg_dist += weight * user["Avg Dist"][j]
        avg_ratio += weight * user["Avg Scale:Dist"][j]

    articleTasks["Avg Dist"][i] = avg_dist
    articleTasks["Avg Ratio"][i] = avg_ratio

articleTasks = articleTasks.sort_values(by= ['Mode', 'Time'])
articleTasks.to_csv("ArticleExperimentData.csv")

taskDf = articleTasks
taskDf
task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
task_n = taskDf.loc[taskDf["Mode"] == "Static"]

print(f"ARTICLE TASKS: ")

taskRAvgTime = task_r['Time'].mean()
taskRStd = task_r['Time'].std()
print(f"responsive tasks Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Time'].mean()
taskNStd = task_n['Time'].std()
print(f"normal tasks Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))


taskRAvgTime = task_r['Guesses'].mean()
taskRStd = task_r['Guesses'].std()
print(f"responsive tasks Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Guesses'].mean()
taskNStd = task_n['Guesses'].std()
print(f"normal tasks Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

taskRAvgTime = task_r['Avg Dist'].mean()
taskRStd = task_r['Avg Dist'].std()
print(f"responsive tasks Avg Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Dist'].mean()
taskNStd = task_n['Avg Dist'].std()
print(f"normal tasks Avg Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))

taskRAvgTime = task_r['Avg Ratio'].mean()
taskRStd = task_r['Avg Ratio'].std()
print(f"responsive tasks Avg Ratio: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Ratio'].mean()
taskNStd = task_n['Avg Ratio'].std()
print(f"normal tasks Avg Ratio: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Ratio'], task_n['Avg Ratio']))

ARTICLE TASKS: 
responsive tasks Time: 35496.000 ± 22396.322
normal tasks Time: 45099.091 ± 25403.666
Ttest_indResult(statistic=-1.346660943843676, pvalue=0.18514493767770876)
responsive tasks Guesses: 5.130 ± 1.792
normal tasks Guesses: 6.318 ± 2.297
Ttest_indResult(statistic=-1.939236524146663, pvalue=0.05905204123368115)
responsive tasks Avg Dist: 0.626 ± 0.044
normal tasks Avg Dist: 0.614 ± 0.042
Ttest_indResult(statistic=0.9238957194188783, pvalue=0.36069633357354125)
responsive tasks Avg Ratio: 0.585 ± 0.060
normal tasks Avg Ratio: 0.585 ± 0.060
Ttest_indResult(statistic=0.02399582515524338, pvalue=0.9809669510451166)


<ipython-input-250-27c31a2d19bd>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articleTasks["Time"][i] = total_time
<ipython-input-250-27c31a2d19bd>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articleTasks["Guesses"][i] = total_guesses
<ipython-input-250-27c31a2d19bd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articleTasks["Application"][i] = "Article"
<ipython-input-250-27c31a2d19bd>:31: SettingWithCopyWarning: 
A val

In [251]:
numpad =  dataFrame.loc[dataFrame["Application"] == "Numpad"]
for i in range(1,3):
    taskDf = numpad.loc[numpad["Task"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
    task_n = taskDf.loc[taskDf["Mode"] == "Static"]

    print(f"Numpad TASK {i}: ")

    taskRAvgTime = task_r['Time'].mean()
    taskRStd = task_r['Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Time'].mean()
    taskNStd = task_n['Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))
    
    taskRAvgTime = task_r['Guesses'].mean()
    taskRStd = task_r['Guesses'].std()
    print(f"responsive task {str(i)} Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Guesses'].mean()
    taskNStd = task_n['Guesses'].std()
    print(f"normal task {str(i)} Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

    taskRAvgTime = task_r['Avg Dist'].mean()
    taskRStd = task_r['Avg Dist'].std()
    print(f"responsive task {str(i)} Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Avg Dist'].mean()
    taskNStd = task_n['Avg Dist'].std()
    print(f"normal task {str(i)} Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))


Numpad TASK 1: 
responsive task 1 Time: 28847.609 ± 13268.076
normal task 1 Time: 24911.773 ± 9271.263
Ttest_indResult(statistic=1.1485301906892023, pvalue=0.25709893555650787)
responsive task 1 Guesses: 1.043 ± 0.209
normal task 1 Guesses: 1.091 ± 0.294
Ttest_indResult(statistic=-0.626117235046913, pvalue=0.5345477403968492)
responsive task 1 Dist: 0.548 ± 0.057
normal task 1 Dist: 0.557 ± 0.081
Ttest_indResult(statistic=-0.3927142627020731, pvalue=0.6964706298033496)
Numpad TASK 2: 
responsive task 2 Time: 53535.391 ± 59750.494
normal task 2 Time: 31114.227 ± 22548.576
Ttest_indResult(statistic=1.650557169528861, pvalue=0.10611550592249368)
responsive task 2 Guesses: 1.261 ± 0.752
normal task 2 Guesses: 1.045 ± 0.213
Ttest_indResult(statistic=1.2944933360701294, pvalue=0.2024037595750517)
responsive task 2 Dist: 0.573 ± 0.090
normal task 2 Dist: 0.562 ± 0.102
Ttest_indResult(statistic=0.38718810412768845, pvalue=0.7005267429659894)


In [252]:
numpadTasks = pandas.DataFrame(columns=['userID', 'Mode', 'Application', 'Time', 'Guesses', 'Avg Dist', 'Avg Ratio' ])
numpadTasks["userID"] = numpad["userID"]
numpadTasks["Mode"] = numpad["Mode"]
numpadTasks = numpadTasks.drop_duplicates(subset=["userID"], keep="first")

for i in numpadTasks.index:
    userID = numpadTasks["userID"][i]
    total_time = 0
    user = numpad.loc[numpad["userID"] == userID]
    total_time = sum(user["Time"])
    total_guesses = sum(user["Guesses"])

    numpadTasks["Time"][i] = total_time
    numpadTasks["Guesses"][i] = total_guesses
    numpadTasks["Application"][i] = "Numpad"

for i in numpadTasks.index:
    userID = numpadTasks["userID"][i]
    user = numpad.loc[numpad["userID"] == userID]
    total_time = numpadTasks["Time"][i]
    avg_dist = 0
    avg_ratio = 0
    for j in user.index:
        task_time = user["Time"][j]
        weight = task_time/total_time
        avg_dist += weight * user["Avg Dist"][j]
        avg_ratio += weight * user["Avg Scale:Dist"][j]

    numpadTasks["Avg Dist"][i] = avg_dist
    numpadTasks["Avg Ratio"][i] = avg_ratio

numpadTasks = numpadTasks.sort_values(by= ['Mode', 'Time'])
numpadTasks.to_csv("NumpadExperimentData.csv")

taskDf = numpadTasks
taskDf
task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
task_n = taskDf.loc[taskDf["Mode"] == "Static"]

print(f"NUMPAD TASKS: ")

taskRAvgTime = task_r['Time'].mean()
taskRStd = task_r['Time'].std()
print(f"responsive tasks Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Time'].mean()
taskNStd = task_n['Time'].std()
print(f"normal tasks Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))


taskRAvgTime = task_r['Guesses'].mean()
taskRStd = task_r['Guesses'].std()
print(f"responsive tasks Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Guesses'].mean()
taskNStd = task_n['Guesses'].std()
print(f"normal tasks Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

taskNAvgTime = task_n['Avg Dist'].mean()
taskNStd = task_n['Avg Dist'].std()
print(f"normal tasks Avg Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

taskNAvgTime = task_n['Avg Dist'].mean()
taskNStd = task_n['Avg Dist'].std()
print(f"normal tasks Avg Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))

taskRAvgTime = task_r['Avg Ratio'].mean()
taskRStd = task_r['Avg Ratio'].std()
print(f"responsive tasks Avg Ratio: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Ratio'].mean()
taskNStd = task_n['Avg Ratio'].std()
print(f"normal tasks Avg Ratio: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Ratio'], task_n['Avg Ratio']))

NUMPAD TASKS: 
responsive tasks Time: 82383.000 ± 59884.352
normal tasks Time: 56026.000 ± 22913.590
Ttest_indResult(statistic=1.9327222234179222, pvalue=0.059873907233486755)
responsive tasks Guesses: 2.304 ± 0.765
normal tasks Guesses: 2.136 ± 0.351
Ttest_indResult(statistic=0.9394209291244621, pvalue=0.35276202948893265)
normal tasks Avg Dist: 0.566 ± 0.081
normal tasks Avg Dist: 0.566 ± 0.081
Ttest_indResult(statistic=-0.11390271866596995, pvalue=0.9098450801023017)
responsive tasks Avg Ratio: 1.341 ± 0.162
normal tasks Avg Ratio: 1.326 ± 0.170
Ttest_indResult(statistic=0.3025842207634834, pvalue=0.7636656631857522)


<ipython-input-252-1c30789e34ca>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numpadTasks["Time"][i] = total_time
<ipython-input-252-1c30789e34ca>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numpadTasks["Guesses"][i] = total_guesses
<ipython-input-252-1c30789e34ca>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numpadTasks["Application"][i] = "Numpad"
<ipython-input-252-1c30789e34ca>:29: SettingWithCopyWarning: 
A value i

In [253]:
weather =  dataFrame.loc[dataFrame["Application"] == "Weather"]
for i in range(1,4):
    taskDf = weather.loc[weather["Task"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
    task_n = taskDf.loc[taskDf["Mode"] == "Static"]

    print(f"Weather TASK {i}: ")

    taskRAvgTime = task_r['Time'].mean()
    taskRStd = task_r['Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Time'].mean()
    taskNStd = task_n['Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))
    
    taskRAvgTime = task_r['Guesses'].mean()
    taskRStd = task_r['Guesses'].std()
    print(f"responsive task {str(i)} Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Guesses'].mean()
    taskNStd = task_n['Guesses'].std()
    print(f"normal task {str(i)} Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

    taskRAvgTime = task_r['Avg Dist'].mean()
    taskRStd = task_r['Avg Dist'].std()
    print(f"responsive task {str(i)} Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Avg Dist'].mean()
    taskNStd = task_n['Avg Dist'].std()
    print(f"normal task {str(i)} Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))

Weather TASK 1: 
responsive task 1 Time: 56057.636 ± 91488.185
normal task 1 Time: 26658.952 ± 13485.498
Ttest_indResult(statistic=1.4567480367843535, pvalue=0.1528023569915703)
responsive task 1 Guesses: 7.955 ± 12.053
normal task 1 Guesses: 5.048 ± 5.491
Ttest_indResult(statistic=1.009331866898274, pvalue=0.31873535675680703)
responsive task 1 Dist: 0.697 ± 0.102
normal task 1 Dist: 0.577 ± 0.045
Ttest_indResult(statistic=4.947774806711382, pvalue=1.32661962396392e-05)
Weather TASK 2: 
responsive task 2 Time: 59949.182 ± 30794.566
normal task 2 Time: 57360.238 ± 68450.821
Ttest_indResult(statistic=0.1611999509384577, pvalue=0.8727276363332076)
responsive task 2 Guesses: 22.500 ± 8.755
normal task 2 Guesses: 22.286 ± 12.207
Ttest_indResult(statistic=0.06638587784263188, pvalue=0.947393280284176)
responsive task 2 Dist: 0.600 ± 0.058
normal task 2 Dist: 0.592 ± 0.050
Ttest_indResult(statistic=0.5135610520937607, pvalue=0.6103155668620123)
Weather TASK 3: 
responsive task 3 Time: 49404.

In [254]:
weatherTasks = pandas.DataFrame(columns=['userID', 'Mode', 'Application', 'Time', 'Guesses', 'Avg Dist', 'Avg Ratio' ])
weatherTasks["userID"] = weather["userID"]
weatherTasks["Mode"] = weather["Mode"]
weatherTasks = weatherTasks.drop_duplicates(subset=["userID"], keep="first")

for i in weatherTasks.index:
    userID = weatherTasks["userID"][i]
    total_time = 0
    user = weather.loc[weather["userID"] == userID]
    total_time = sum(user["Time"])
    total_guesses = sum(user["Guesses"])

    weatherTasks["Time"][i] = total_time
    weatherTasks["Guesses"][i] = total_guesses
    weatherTasks["Application"][i] = "Weather"

for i in weatherTasks.index:
    userID = weatherTasks["userID"][i]
    user = weather.loc[weather["userID"] == userID]
    total_time = weatherTasks["Time"][i]
    avg_dist = 0
    avg_ratio = 0
    for j in user.index:
        task_time = user["Time"][j]
        weight = task_time/total_time
        avg_dist += weight * user["Avg Dist"][j]
        avg_ratio += weight * user["Avg Scale:Dist"][j]

    weatherTasks["Avg Dist"][i] = avg_dist
    weatherTasks["Avg Ratio"][i] = avg_ratio

weatherTasks = weatherTasks.sort_values(by= ['Mode', 'Time'])
weatherTasks.to_csv("WeatherExperimentData.csv")

taskDf = weatherTasks
taskDf
task_r = taskDf.loc[taskDf["Mode"] == "Responsive"]
task_n = taskDf.loc[taskDf["Mode"] == "Static"]

print(f"WEATHER TASKS: ")

taskRAvgTime = task_r['Time'].mean()
taskRStd = task_r['Time'].std()
print(f"responsive tasks Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Time'].mean()
taskNStd = task_n['Time'].std()
print(f"normal tasks Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Time'], task_n['Time']))


taskRAvgTime = task_r['Guesses'].mean()
taskRStd = task_r['Guesses'].std()
print(f"responsive tasks Guesses: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Guesses'].mean()
taskNStd = task_n['Guesses'].std()
print(f"normal tasks Guesses: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

taskRAvgTime = task_r['Avg Dist'].mean()
taskRStd = task_r['Avg Dist'].std()
print(f"responsive tasks Avg Dist: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Dist'].mean()
taskNStd = task_n['Avg Dist'].std()
print(f"normal tasks Avg Dist: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Dist'], task_n['Avg Dist']))

taskRAvgTime = task_r['Avg Ratio'].mean()
taskRStd = task_r['Avg Ratio'].std()
print(f"responsive tasks Avg Ratio: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Avg Ratio'].mean()
taskNStd = task_n['Avg Ratio'].std()
print(f"normal tasks Avg Ratio: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Avg Ratio'], task_n['Avg Ratio']))

<ipython-input-254-99c9423c7868>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weatherTasks["Time"][i] = total_time
<ipython-input-254-99c9423c7868>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weatherTasks["Guesses"][i] = total_guesses
<ipython-input-254-99c9423c7868>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weatherTasks["Application"][i] = "Weather"
<ipython-input-254-99c9423c7868>:29: SettingWithCopyWarning: 
A val

WEATHER TASKS: 
responsive tasks Time: 165411.727 ± 105099.860
normal tasks Time: 127929.810 ± 84347.194
Ttest_indResult(statistic=1.285928831791032, pvalue=0.2056831687677855)
responsive tasks Guesses: 44.682 ± 27.753
normal tasks Guesses: 38.905 ± 17.254
Ttest_indResult(statistic=0.815095492541208, pvalue=0.41972743373007315)
responsive tasks Avg Dist: 0.636 ± 0.069
normal tasks Avg Dist: 0.608 ± 0.044
Ttest_indResult(statistic=1.6113917293058375, pvalue=0.11476711221941983)
responsive tasks Avg Ratio: 0.799 ± 0.194
normal tasks Avg Ratio: 0.816 ± 0.122
Ttest_indResult(statistic=-0.345561523496962, pvalue=0.7314390653758156)


In [ ]:
responsive = dataFrame.loc[dataFrame["Mode"] == "Responsive"]
responsive

visualAngleDf = pandas.DataFrame(columns=['userID', 'Application' , 'Task', 'Time', 'Guesses', 'Time Spent in VA1', 'Time Spent in VA2', 'Time Spent in VA3', 'Time Spent in VA4', 'Time Spent in VA5', 'Time Spent in VA6', 'Time Spent in VA7', 'Time Spent in VA8', 'Time Spent in VA9', 'Default VA' ])

visualAngleDf["userID"] = responsive["userID"]
visualAngleDf["Task"] = responsive["Task"]
visualAngleDf["Application"] = responsive["Application"]
visualAngleDf["Time"] = responsive["Time"]
visualAngleDf["Guesses"] = responsive["Guesses"]

mask = visualAngleDf["Time"] > 0

visualAngleDf.loc[mask, 'Time Spent in VA1'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA2'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA3'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA4'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA5'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA6'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA7'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA8'] = 0
visualAngleDf.loc[mask, 'Time Spent in VA9'] = 0
visualAngleDf.loc[mask, 'Default VA'] = 0

In [ ]:
for i in visualAngleDf.index:
    userID = visualAngleDf["userID"][i]
    task = visualAngleDf["Task"][i]
    application = visualAngleDf["Application"][i]
    task_num = task - 1

    if(application == "Numpad"):
        task_num += 4

    if(application == "Weather"):
        task_num += 6
    #need math to convert task and app to task_number

    responsiveSession = sessionsDf.loc[sessionsDf["userID"] == userID]

    startEvent = startEvents.loc[(startEvents["userID"] == userID) & (startEvents["task_number"] == task_num)]
    startTime = startEvent["unixTime"].to_numpy()[0]
    endEvent = endEvents.loc[(endEvents["userID"] == userID) & (endEvents["task_number"] == task_num)]
    endTime = endEvent["unixTime"].to_numpy()[0]
    framesUnpacked = list(itertools.chain(*responsiveSession["frames"]))
    sessionFramesDf = pandas.DataFrame(framesUnpacked)

    sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] >= startTime]
    #sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] <= endTime + 100]
    sessionFramesDf = sessionFramesDf.sort_values("unixTime")

    lastFrame = sessionFramesDf.loc[sessionFramesDf["experimentEvents"].str.len() != 0]
    lastFrame = lastFrame.loc[lastFrame["unixTime"] >= endTime]
    lastFrame = lastFrame.loc[lastFrame["unixTime"] <= endTime + 600]

    if(lastFrame.shape[0] != 1):
        print(endTime)
        print(lastFrame.shape[0])
        print(lastFrame["unixTime"].to_numpy())

    lastFrameTime = lastFrame["unixTime"].to_numpy()[0]
    sessionFramesDf = sessionFramesDf.loc[sessionFramesDf["unixTime"] <= lastFrameTime]

    responsiveData = pandas.DataFrame(list(itertools.chain(sessionFramesDf["responsiveData"])))

    ratio = pandas.DataFrame(columns=['ratio', 'unixTime'])
    ratio['ratio'] = responsiveData['ratio']
    ratio["unixTime"] = sessionFramesDf["unixTime"].to_numpy()

    mask = (visualAngleDf["userID"] == userID) & (visualAngleDf["Task"] == task) & (visualAngleDf["Application"] == application)
    total_time = 0

    if application == "Article":
        VA1 = .35523
        VA2 = .333333
        VA3 = .4736839
        VA4 = .64285685

        if task == 1:
            visualAngleDf.loc[mask, "Default VA"] = "VA2"
        if task == 2:
            visualAngleDf.loc[mask, "Default VA"] = "VA2"
        if task == 3:
            visualAngleDf.loc[mask, "Default VA"] = "VA3"
        if task == 4:
            visualAngleDf.loc[mask, "Default VA"] = "VA4"

        for i in ratio.index:
            if i > 0:
                time = ratio["unixTime"][i] - ratio["unixTime"][i - 1]
            else:
                time = ratio["unixTime"][i] - startTime
            total_time += time
            if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
                visualAngleDf.loc[mask, "Time Spent in VA1"] += time
            if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
                visualAngleDf.loc[mask, "Time Spent in VA2"] += time
            if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
                visualAngleDf.loc[mask, "Time Spent in VA3"] += time
            if(ratio['ratio'][i] >= VA4):
                visualAngleDf.loc[mask, "Time Spent in VA4"] += time

        time = endTime - ratio["unixTime"][i]
        total_time += time
        if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
            visualAngleDf.loc[mask, "Time Spent in VA1"] += time
        if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
            visualAngleDf.loc[mask, "Time Spent in VA2"] += time
        if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
            visualAngleDf.loc[mask, "Time Spent in VA3"] += time
        if(ratio['ratio'][i] >= VA4):
            visualAngleDf.loc[mask, "Time Spent in VA4"] += time


    if application == "Numpad":
        VA1 = .27497517
        VA2 = .384615
        VA3 = .40723984
        VA4 = 1.0442047
        VA5 = 1.20192313

        if task == 1:
            visualAngleDf.loc[mask, "Default VA"] = "VA3"
        if task == 2:
            visualAngleDf.loc[mask, "Default VA"] = "VA5"

        for i in ratio.index:
            if i > 0:
                time = ratio["unixTime"][i] - ratio["unixTime"][i - 1]
            else:
                time = ratio["unixTime"][i] - startTime
            total_time += time
            if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
                visualAngleDf.loc[mask, "Time Spent in VA1"] += time
            if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
                visualAngleDf.loc[mask, "Time Spent in VA2"] += time
            if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
                visualAngleDf.loc[mask, "Time Spent in VA3"] += time
            if(ratio['ratio'][i] < VA5 and ratio['ratio'][i] >= VA4):
                visualAngleDf.loc[mask, "Time Spent in VA4"] += time
            if(ratio['ratio'][i] >= VA5):
                visualAngleDf.loc[mask, "Time Spent in VA5"] += time
        time = endTime - ratio["unixTime"][i]
        total_time += time
        if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
            visualAngleDf.loc[mask, "Time Spent in VA1"] += time
        if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
            visualAngleDf.loc[mask, "Time Spent in VA2"] += time
        if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
            visualAngleDf.loc[mask, "Time Spent in VA3"] += time
        if(ratio['ratio'][i] < VA5 and ratio['ratio'][i] >= VA4):
            visualAngleDf.loc[mask, "Time Spent in VA4"] += time
        if(ratio['ratio'][i] >= VA5):
            visualAngleDf.loc[mask, "Time Spent in VA5"] += time

    if application == "Weather":
        VA1 = .312891085
        VA2 = .312500
        VA3 = .425170065
        VA4 = .62500
        VA5 = .744047
        VA6 = .961538
        VA7 = 1.1363636
        VA8 = 1.19047618
        VA9 = 1.470588287

        if task == 1:
            visualAngleDf.loc[mask, "Default VA"] = "VA3"
        if task == 2:
            visualAngleDf.loc[mask, "Default VA"] = "VA5"
        if task == 3:
            visualAngleDf.loc[mask, "Default VA"] = "VA8"

        for i in ratio.index:
            if i > 0:
                time = ratio["unixTime"][i] - ratio["unixTime"][i - 1]
            else:
                time = ratio["unixTime"][i] - startTime
            total_time += time
            if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
                visualAngleDf.loc[mask, "Time Spent in VA1"] += time
            if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
                visualAngleDf.loc[mask, "Time Spent in VA2"] += time
            if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
                visualAngleDf.loc[mask, "Time Spent in VA3"] += time
            if(ratio['ratio'][i] < VA5 and ratio['ratio'][i] >= VA4):
                visualAngleDf.loc[mask, "Time Spent in VA4"] += time
            if(ratio['ratio'][i] < VA6 and ratio['ratio'][i] >= VA5):
                visualAngleDf.loc[mask, "Time Spent in VA5"] += time
            if(ratio['ratio'][i] < VA7 and ratio['ratio'][i] >= VA6):
                visualAngleDf.loc[mask, "Time Spent in VA6"] += time
            if(ratio['ratio'][i] < VA8 and ratio['ratio'][i] >= VA7):
                visualAngleDf.loc[mask, "Time Spent in VA7"] += time
            if(ratio['ratio'][i] < VA9 and ratio['ratio'][i] >= VA8):
                visualAngleDf.loc[mask, "Time Spent in VA8"] += time
            if(ratio['ratio'][i] >= VA9):
                visualAngleDf.loc[mask, "Time Spent in VA9"] += time

        time = endTime - ratio["unixTime"][i]
        total_time += time
        if((ratio['ratio'][i] < VA2) and (ratio['ratio'][i] >= VA1)):
            visualAngleDf.loc[mask, "Time Spent in VA1"] += time
        if(ratio['ratio'][i] < VA3 and ratio['ratio'][i] >= VA2):
            visualAngleDf.loc[mask, "Time Spent in VA2"] += time
        if(ratio['ratio'][i] < VA4 and ratio['ratio'][i] >= VA3):
            visualAngleDf.loc[mask, "Time Spent in VA3"] += time
        if(ratio['ratio'][i] < VA5 and ratio['ratio'][i] >= VA4):
            visualAngleDf.loc[mask, "Time Spent in VA4"] += time
        if(ratio['ratio'][i] < VA6 and ratio['ratio'][i] >= VA5):
            visualAngleDf.loc[mask, "Time Spent in VA5"] += time
        if(ratio['ratio'][i] < VA7 and ratio['ratio'][i] >= VA6):
            visualAngleDf.loc[mask, "Time Spent in VA6"] += time
        if(ratio['ratio'][i] < VA8 and ratio['ratio'][i] >= VA7):
            visualAngleDf.loc[mask, "Time Spent in VA7"] += time
        if(ratio['ratio'][i] < VA9 and ratio['ratio'][i] >= VA8):
            visualAngleDf.loc[mask, "Time Spent in VA8"] += time
        if(ratio['ratio'][i] >= VA9):
            visualAngleDf.loc[mask, "Time Spent in VA9"] += time

    if(total_time != visualAngleDf.loc[mask, "Time"].to_numpy()[0]):
        print("AHHHHHH")
        print(f"userid {userID}")
        print(f"app {application}")
        print(f"task {task}")
        print(f"total_time: {total_time}")
        print(visualAngleDf.loc[mask, "Time"].to_numpy()[0])



In [ ]:

visualAngleDf = visualAngleDf.sort_values(by=['Application', 'Task', 'Time'])
visualAngleDf.to_csv("responsiveARTimeSpentInVA.csv")